# **Computer Vision 이상치 탐지 알고리즘 경진대회**

start : 220401




# import module

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch
import os
from tqdm import tqdm
from glob import glob
import cv2
import numpy as np
import pandas as pd
import PIL 
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import torchvision as tv
from PIL import Image

# 환경 설정

In [ ]:
from google.colab import drive
drive.mount._DEBUG = True
drive.mount('/content/drive', force_remount=True)

unset HISTFILE; export PS1="root@b2040528c134-25a4d7fca42c422599b593daab303174: "
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
/content# root@b2040528c134-25a4d7fca42c422599b593daab303174: umount -f /content/drive || umount /content/drive; pkill -9 -x drive
umount: /content/drive: no mount point specified.
umount: /content/drive: no mount point specified.
root@b2040528c134-25a4d7fca42c422599b593daab303174: pkill -9 -f /opt/google/drive/directoryprefetcher_binary
root@b2040528c134-25a4d7fca42c422599b593daab303174: ( while `sleep 0.5`; do if [[ -d "/content/drive" && "$(ls -A /content/drive)" != "" ]]; then echo "google.colab.drive MOUNTED"; break; fi; done ) &
[1] 156
root@b2040528c134-25a4d7fca42c422599b593daab303174: ( /opt/google/drive/drive --features=fuse_max_background:1000,max_read_qps:1000,max_write_qps:1000,max_operation_batch_size:15,max_parallel_push_task_instances:10,opendir_timeout_ms:120000,virtual_folders_

In [ ]:
os.chdir('./drive/MyDrive/이상치/')

In [ ]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
    
device

'cuda'

# load data

In [ ]:
# %%time
# !unzip "./open.zip" -d "/content/drive/MyDrive/Dacon/Computer Vision 이상치 탐지 알고리즘 경진대회/data"

In [ ]:
# %%time
# !unzip "./data/train.zip" -d "/content/drive/MyDrive/이상치/data"

In [ ]:
%%time
!unzip "./data/test.zip" -d "/content/drive/MyDrive/이상치/data"

Archive:  ./data/test.zip
replace /content/drive/MyDrive/이상치/data/test/20000.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: /content/drive/MyDrive/이상치/data/test/20000.png  
  inflating: /content/drive/MyDrive/이상치/data/test/20001.png  
  inflating: /content/drive/MyDrive/이상치/data/test/20002.png  
  inflating: /content/drive/MyDrive/이상치/data/test/20003.png  
  inflating: /content/drive/MyDrive/이상치/data/test/20004.png  
  inflating: /content/drive/MyDrive/이상치/data/test/20005.png  
  inflating: /content/drive/MyDrive/이상치/data/test/20006.png  
  inflating: /content/drive/MyDrive/이상치/data/test/20007.png  
  inflating: /content/drive/MyDrive/이상치/data/test/20008.png  
  inflating: /content/drive/MyDrive/이상치/data/test/20009.png  
  inflating: /content/drive/MyDrive/이상치/data/test/20010.png  
  inflating: /content/drive/MyDrive/이상치/data/test/20011.png  
  inflating: /content/drive/MyDrive/이상치/data/test/20012.png  
  inflating: /content/drive/MyDrive/이상치/data/test/20013.png  
  inflatin

In [ ]:
train_x = sorted(glob('./data/train/*.png'))
test = sorted(glob('./data/test/*.png'))
train_csv = pd.read_csv("./data/train_df.csv")
train_label = train_csv["label"]

# dataset

In [ ]:
label_unique = sorted(np.unique(train_label))

label_unique_dir = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_Y = [label_unique_dir[k] for k in train_label]

In [ ]:
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, x_dir,transform=None):
        super().__init__()
        self.transforms = transform
        self.x_img = x_dir 

    def __len__(self):
        return len(self.x_img)

    def __getitem__(self, idx):
        x_img = self.x_img[idx]

        x_img = cv2.imread(x_img)
        x_img = cv2.cvtColor(x_img, cv2.COLOR_BGR2RGB)

        if self.transforms:
            augmented = self.transforms(image=x_img)
            x_img = augmented['image']

        return x_img

In [ ]:
! pip install albumentations==0.4.6

     |████████████████████████████████| 117 kB 8.6 MB/s 
     |████████████████████████████████| 948 kB 41.7 MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.6-py3-none-any.whl size=65174 sha256=cd92579eb3ced5f66f8298a2f3b875e61165bc1974f790634f91cb16fe9346f3
  Stored in directory: /root/.cache/pip/wheels/cf/34/0f/cb2a5f93561a181a4bcc84847ad6aaceea8b5a3127469616cc
Successfully built albumentations
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [ ]:
import albumentations
import albumentations.pytorch

In [ ]:
def transform_inputsize(inputsize):
  return albumentations.Compose([
      albumentations.Resize(inputsize, inputsize),
      albumentations.augmentations.transforms.Normalize(mean=(0.5,), std=(0.5,), p=1.0),
      albumentations.pytorch.transforms.ToTensorV2(p=1.0)
      ])

In [ ]:
batch_size=32
test_dataset224 = TestDataset(test,transform_inputsize(224))
test_loader224 = torch.utils.data.DataLoader(test_dataset224, shuffle=False, batch_size=batch_size)
test_dataset300 = TestDataset(test,transform_inputsize(300))
test_loader300 = torch.utils.data.DataLoader(test_dataset300, shuffle=False, batch_size=batch_size)

# 모델 정의

In [ ]:
!pip install timm
import timm

     |████████████████████████████████| 431 kB 7.6 MB/s 


In [ ]:
swin_tiny_patch4_window7_224 = timm.create_model('swin_tiny_patch4_window7_224',pretrained=True,num_classes=88,in_chans=3)
swin_tiny_patch4_window7_224.load_state_dict(torch.load('swin_tiny_patch4_window7_224.pt'))

/usr/local/lib/python3.7/dist-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Downloading: "https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_tiny_patch4_window7_224.pth" to /root/.cache/torch/hub/checkpoints/swin_tiny_patch4_window7_224.pth


<All keys matched successfully>

In [ ]:
!pip install efficientnet_pytorch
from efficientnet_pytorch import EfficientNet
efficientnetB2 = EfficientNet.from_pretrained('efficientnet-b2', num_classes=88)
efficientnetB2.load_state_dict(torch.load('efficientnet-b2.pt'))

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=14afc985da3858e17bb49cb20fd11b8df0c2822b3b6ff2b8124c471accaa1b63
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b2-8bb594d6.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b2-8bb594d6.pth


  0%|          | 0.00/35.1M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b2


<All keys matched successfully>

In [ ]:
efficientnetB0 = EfficientNet.from_pretrained('efficientnet-b0', num_classes=88)
efficientnetB0.load_state_dict(torch.load('efficientnet-b0.pt'))

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth


  0%|          | 0.00/20.4M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b0


<All keys matched successfully>

In [ ]:
mixnet_s=timm.create_model('mixnet_s',pretrained=True,num_classes=88,in_chans=3)
mixnet_s.load_state_dict(torch.load('mixnet_s.pt'))

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/mixnet_s-a907afbc.pth" to /root/.cache/torch/hub/checkpoints/mixnet_s-a907afbc.pth


<All keys matched successfully>

# Inference v1

In [ ]:
model.to(device)
model.eval()
f_pred = []

with torch.no_grad():
    for x,_ in tqdm(test_loader):
        x = x.to(device)
        pred = model(x)
        f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist())

  0%|          | 0/135 [00:00<?, ?it/s]

# Inference v2 (TTA)

In [ ]:
! pip install git+https://github.com/qubvel/ttach

  Cloning https://github.com/qubvel/ttach to /tmp/pip-req-build-42zfufyx
  Running command git clone -q https://github.com/qubvel/ttach /tmp/pip-req-build-42zfufyx
  Created wheel for ttach: filename=ttach-0.0.3-py3-none-any.whl size=9910 sha256=072c3102ebb2973617a9469047a2dc67d38efbcb036a2214a6ed9adede61420f
  Stored in directory: /tmp/pip-ephem-wheel-cache-7j7nzdyh/wheels/eb/80/fe/a9da0a4107266ca9ee6022362bb6e28d6abcb946ececb1630e
Successfully built ttach


In [ ]:
import ttach as tta

tta_transforms = tta.Compose(
    [
        tta.Rotate90(angles=[0, 180]),
        tta.Multiply(factors=[0.9, 1, 1.1]),
    ]
)

swin_tiny_patch4_window7_224 = tta.ClassificationTTAWrapper(swin_tiny_patch4_window7_224, tta_transforms)
tta_seresnext50_32x4d = tta.ClassificationTTAWrapper(seresnext50_32x4d, tta_transforms) 
tta_efficientnetB2 = tta.ClassificationTTAWrapper(efficientnetB2, tta_transforms) 
tta_efficientnetB0 = tta.ClassificationTTAWrapper(efficientnetB0, tta_transforms) 

In [ ]:
tta_model1.to(device)
tta_model1.eval()
# tta_model2.to(device)
# tta_model2.eval()
# tta_model3.to(device)
# tta_model3.eval()
tta_model4.to(device)
tta_model4.eval()
# tta_model5.to(device)
# tta_model5.eval()
# tta_model6.to(device)
# tta_model6.eval()
tta_model7.to(device)
tta_model7.eval()
predd1 = []
predd4 = []
predd5 = []
predd7 = []
f_pred = []

with torch.no_grad():
    for x,_ in tqdm(test_loader):
        x = x.to(device)
        pred1 = tta_model1(x)
        # pred2 = tta_model2(x)
        # pred3 = tta_model3(x)
        pred4 = tta_model4(x)
        # pred5 = tta_model5(x)
        # pred6 = tta_model6(x)
        pred7 = tta_model7(x)
        pred=(pred7+pred4+pred1)/3
        f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist())
        # predd1.extend(pred1.detach().cpu().numpy().tolist())
        # predd4.extend(pred4.detach().cpu().numpy().tolist())
        # predd5.extend(pred5.detach().cpu().numpy().tolist())
        # predd7.extend(pred7.detach().cpu().numpy().tolist())


  0%|          | 0/36 [00:00<?, ?it/s]

# Inference v3(TTA+difference input size)

In [ ]:
! pip install git+https://github.com/qubvel/ttach

  Cloning https://github.com/qubvel/ttach to /tmp/pip-req-build-3ilbv6tn
  Running command git clone -q https://github.com/qubvel/ttach /tmp/pip-req-build-3ilbv6tn
  Created wheel for ttach: filename=ttach-0.0.3-py3-none-any.whl size=9910 sha256=dae7fb0da842609f7575c5634cbe8ed023aa92b298fb54c926d7a4899db74a3b
  Stored in directory: /tmp/pip-ephem-wheel-cache-xp9kl5zl/wheels/eb/80/fe/a9da0a4107266ca9ee6022362bb6e28d6abcb946ececb1630e
Successfully built ttach


In [ ]:
import ttach

In [ ]:
#ttach.Scale함수는 input output 크기가 달라서 같게 함수 수정
class myScale(ttach.Scale):
    def apply_aug_image(self, image, scale=1, **kwargs):
        image_h,image_w=image.shape[2], image.shape[3]
        if scale != self.identity_param:
            image = ttach.functional.scale(
                image,
                scale,
                interpolation=self.interpolation,
                align_corners=self.align_corners,
            )
            if image.shape[2]>=image_h:
                image=ttach.functional.center_crop(image, image_h, image_w)
            else :
                image=ttach.functional.resize(image, (image_h,image_w))
        return image

In [ ]:
tta_transforms = ttach.Compose(
    [
        ttach.Rotate90(angles=[0, 90, 180, 270]),
        ttach.Multiply(factors=[0.9, 1, 1.1]),
        myScale(scales=[0.9,1, 1.1])
    ]
)

tta_swin_tiny_patch4_window7_224 = ttach.ClassificationTTAWrapper(swin_tiny_patch4_window7_224, tta_transforms)
tta_mixnet_s = ttach.ClassificationTTAWrapper(mixnet_s, tta_transforms) 
tta_efficientnetB2 = ttach.ClassificationTTAWrapper(efficientnetB2, tta_transforms) 
tta_efficientnetB0 = ttach.ClassificationTTAWrapper(efficientnetB0, tta_transforms) 

In [ ]:
tta_swin_tiny_patch4_window7_224.to(device)
tta_swin_tiny_patch4_window7_224.eval()
tta_mixnet_s.to(device)
tta_mixnet_s.eval()
tta_efficientnetB2.to(device)
tta_efficientnetB2.eval()
tta_efficientnetB0.to(device)
tta_efficientnetB0.eval()
f_pred = []

with torch.no_grad():
    for img224,img300 in iter(zip(test_loader224,test_loader300)):

        img224 = img224.to(device)
        img300 = img300.to(device)

        pred1 = tta_swin_tiny_patch4_window7_224(img224)
        # pred2 = tta_efficientnet_b1_pruned(img300)
        pred3 = tta_efficientnetB2(img300)
        pred4 = tta_efficientnetB0(img300)
        pred5 = tta_mixnet_s(img300)

        pred = (pred1+pred5+pred3+pred4)/4
        
        f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist())

# Make Submission 

In [ ]:
label_decoder = {val:key for key, val in label_unique_dir.items()}

f_result = [label_decoder[result] for result in f_pred]

In [ ]:
submission = pd.read_csv("./data/sample_submission.csv")

submission["label"] = f_result

submission

,index,label
0,0,tile-glue_strip
1,1,grid-good
2,2,transistor-good
3,3,tile-gray_stroke
4,4,tile-good
...,...,...
2149,2149,tile-gray_stroke
2150,2150,screw-good
2151,2151,grid-good
2152,2152,cable-good


In [ ]:
submission.to_csv("swin_tiny_patch4_window7_224+efficientnet-b2+efficientnet-b0+mixnet_s__2.csv", index = False)